In [2]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
import re
from debug_dataset import debug_dataset
import pickle
from datasets import load_dataset
import yaml
import datetime
from multiprocessing import Pool, cpu_count

In [3]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [ ]:
if config['dataset']['DEBUG']:   
    dataset = debug_dataset
else:
    dataset = load_dataset('0x7o/taiga', split='train')['text'].to_list()

Resolving data files:   0%|          | 0/117 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/117 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/116 [00:00<?, ?it/s]

In [8]:
type(dataset)

datasets.arrow_dataset.Column